# Data

1. Gets necessary data from CRSP for S&P constituents
2. Transforms PDFs of FOMC statements to URLs (so that all FOMCs are now in a url)
3. Scrapes necessary text from all URLs
4. Removes noise from FOMC statements

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import wrds

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nathanueda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Step 1: Get S&P Data

In [30]:
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [31]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        and b.date<='12/31/2023'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [32]:
sp500 = sp500.drop(columns=['start', 'ending'])

In [33]:
sp500

,permno,date,ret
0,64936,2000-01-03,-0.028662
1,24205,2000-01-03,-0.036496
2,60441,2000-01-03,-0.028926
3,45751,2000-01-03,-0.011757
4,76887,2000-01-03,-0.042553
...,...,...,...
28546,87445,2023-12-29,0.004683
28547,21792,2023-12-29,0.000350
28548,13356,2023-12-29,0.002258
28549,58819,2023-12-29,-0.000390


In [34]:
sp500.to_pickle('data/sp500.pkl')

In [35]:
sp500.isna().sum()

permno      0
date        0
ret       101
dtype: int64

In [36]:
(sp500[sp500['ret'].isna()])['permno'].nunique()

98

### Step 2: Scrape Relevant Text from URLs

In [69]:
# fomc_urls = pd.read_csv('data/fomc statement times pre 2011 with urls.csv')

# has original urls and urls from pdfs
fomc_urls = pd.read_csv('data/all_fomc_urls.csv')
# fomc_urls = fomc_urls.drop(columns=['Statement Time', 'Press Conference'])
fomc_urls = fomc_urls.rename(columns={'Statement Date': 'date', 'URL':'url'})
fomc_urls['date'] = pd.to_datetime(fomc_urls['date'])
fomc_urls

,date,url
0,1999-05-18,https://www.federalreserve.gov/boarddocs/press...
1,1999-06-30,https://www.federalreserve.gov/boarddocs/press...
2,1999-08-24,https://www.federalreserve.gov/boarddocs/press...
3,1999-10-05,https://www.federalreserve.gov/boarddocs/press...
4,1999-11-16,https://www.federalreserve.gov/boarddocs/press...
...,...,...
162,2023-09-20,https://www.federalreserve.gov/newsevents/pres...
163,2023-11-01,https://www.federalreserve.gov/newsevents/pres...
164,2023-12-13,https://www.federalreserve.gov/newsevents/pres...
165,2024-01-31,https://www.federalreserve.gov/newsevents/pres...


In [70]:
fomc_statements = []

for index, row in fomc_urls.iterrows():
    date = row['date']
    url = row['url']
    
    response = requests.get(url)
    
    # request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        statement_text = soup.get_text(separator='\n', strip=True)
                
        content_div = soup.find('div', class_='col-xs-12 col-sm-8 col-md-8')

        # dates in range: 01-31-2006 - 01-26-2011 enter here and store relevant text in col-xs-12 col-sm-8 col-md-8
        # converted pdfs to urls as well which follow this format
        if content_div:
            paragraphs = content_div.find_all('p')
            statement_text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
        
        # dates in range: 05-18-1999 - 12-13-2005 enter here and store relevant text in <p>
        else:
            statement_text= soup.find('p').get_text(separator='\n', strip=True)
            
        # add statment
        fomc_statements.append({'date': date, 'statement': statement_text})
    else:
        print(f"Failed to retrieve URL at index {index}: {url}")

In [71]:
fomc_statements = pd.DataFrame(fomc_statements)

print("Statements Format 1 (05-18-1999 - 12-13-2005):")
display(fomc_statements['statement'].iloc[0])

print("Statements Format 2 (01-31-2006 - 01-26-2011) and onwards:")

display(fomc_statements['statement'].iloc[-1])

Statements Format 1 (05-18-1999 - 12-13-2005):


"For immediate release\nThe Federal Reserve released the following statement after today's Federal Open Market Committee meeting:\nWhile the FOMC did not take action today to alter the stance of monetary policy, the Committee was concerned about the potential for a buildup of inflationary imbalances that could undermine the favorable performance of the economy and therefore adopted a directive that is tilted toward the possibility of a  firming in the stance of monetary policy.  Trend increases in costs and core prices have generally remained quite subdued.  But domestic financial markets have recovered and foreign economic prospects have improved since the easing of monetary policy last fall.  Against the background of already-tight domestic labor markets and ongoing strength in demand in excess of productivity gains, the Committee recognizes the need to be alert to developments over coming months that might indicate that financial conditions may no longer be consistent with containin

Statements Format 2 (01-31-2006 - 01-26-2011) and onwards:


"Recent indicators suggest that economic activity has been expanding at a solid pace. Job gains have remained strong, and the unemployment rate has remained low. Inflation has eased over the past year but remains elevated.\nThe Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. The Committee judges that the risks to achieving its employment and inflation goals are moving into better balance. The economic outlook is uncertain, and the Committee remains highly attentive to inflation risks.\nIn support of its goals, the Committee decided to maintain the target range for the federal funds rate at 5-1/4 to 5-1/2 percent. In considering any adjustments to the target range for the federal funds rate, the Committee will carefully assess incoming data, the evolving outlook, and the balance of risks. The Committee does not expect it will be appropriate to reduce the target range until it has gained greater confidence that inflation is moving

### Step 3: Remove Noise in FOMC Statements

In [88]:
def remove_noise(text):

    text = text.lower()

    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]

    cleaned_text = ' '.join(tokens)

    return cleaned_text

fomc_statements['cleaned_statement'] = fomc_statements['statement'].apply(remove_noise)


In [89]:
fomc_statements.to_pickle('data/fomc_statements.pkl')